Dash renders web applications as a "single-page app". This means that the application does not completely reload when the user navigates the application, making browsing very fast.

There are two components that aid page navigation:
**dcc.Location** and **dcc.Link**.

### dcc.Location

represents the location bar in your web browser through the ``pathname`` property.

### dcc.Link

The Link element updates the pathname of the browser without refreshing the page. 

### Structure a multi-page app

Here's how to structure a multi-page app, where each app is contained in a separate file.

File structure:

- app.py
- datasetfolder
- index.py
- apps
    - app1.py
    - app2.py
    
    
In app.py, write the following code:
```
import dash
import pandas as pd
app = dash.Dash(__name__, suppress_callback_exceptions=True)
server = app.server
dataset = pd.read_csv('corona dataset Pakistan/PK COVID-19-3jun.csv')


```

index.py loads different apps on different urls like this:
```
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

from app import app
from apps import app1, app2


app.layout = html.Div([
    dcc.Location(id='url'),
	dcc.Link('Go to App 1', href='/apps/app1'),
	html.Hr(),
	dcc.Link('Go to App 2', href='/apps/app2'),
    html.Div(id='page-content')
])


@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/apps/app1':
        return app1.layout
    elif pathname == '/apps/app2':
        return app2.layout
    else:
        return 'This is the home screen'

if __name__ == '__main__':
    app.run_server(debug=True)
```

In app1.py

```
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from app import app,dataset
import plotly.express as px
import pandas as pd
import numpy as np

dataset['Date'] = pd.to_datetime(dataset['Date'])
datewise_add = dataset.pivot_table(index=['Date'],aggfunc=np.sum)

def recovered_cases():
	fig = px.line(datewise_add, x=datewise_add.index, y=datewise_add['Recovered'], title="Trend analysis of Recovered Cases") 
	return fig


layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Page 1', style = {'textAlign':'center'}),
    dcc.Graph(id = 'line_plot1', figure = recovered_cases()),
	])


```

In app2.py, write the following code:
```
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from app import app,dataset
import plotly.express as px
import pandas as pd
import numpy as np

dataset['Date'] = pd.to_datetime(dataset['Date'])
datewise_add = dataset.pivot_table(index=['Date'],aggfunc=np.sum)

def death_cases():
	fig = px.line(datewise_add, x=datewise_add.index, y=datewise_add['Deaths'], title="Trend analysis of Death Cases") 
	return fig


layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Page 2', style = {'textAlign':'center'}),
    dcc.Graph(id = 'line_plot1', figure = death_cases()),
	])



```

It is worth noting that the Dash instance is defined in a separate **app.py**, while the entry point for running the app is **index.py**. This separation is required to avoid circular imports: the files containing the callback definitions require access to the Dash app instance however if this were imported from index.py, the initial loading of index.py would ultimately require itself to be already imported, which cannot be satisfied.